# Importing Libraries

In [335]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import zipfile
import re
import warnings
warnings.filterwarnings('ignore')

from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler

# Reading Data

In [4]:
#Reading the zipped file

zf = zipfile.ZipFile(r'C:\Users\nishantjairath\Downloads\drugsCom_raw.zip')
#zf_label = zipfile.ZipFile(r'C:\Users\nishantjairath\Downloads\train_labels.csv.zip')

In [5]:
#reading zipped file contents

print(zf.namelist())

['drugsComTest_raw.tsv', 'drugsComTrain_raw.tsv']


In [481]:
df = pd.read_csv(zf.open('drugsComTrain_raw.tsv'),sep='\t')
df_label = pd.read_csv(zf.open('drugsComTest_raw.tsv'),sep='\t')

In [9]:
df.head()

,Unnamed: 0,drugName,condition,review,rating,date,usefulCount
0,206461,Valsartan,Left Ventricular Dysfunction,"""It has no side effect, I take it in combinati...",9.0,"May 20, 2012",27
1,95260,Guanfacine,ADHD,"""My son is halfway through his fourth week of ...",8.0,"April 27, 2010",192
2,92703,Lybrel,Birth Control,"""I used to take another oral contraceptive, wh...",5.0,"December 14, 2009",17
3,138000,Ortho Evra,Birth Control,"""This is my first time using any form of birth...",8.0,"November 3, 2015",10
4,35696,Buprenorphine / naloxone,Opiate Dependence,"""Suboxone has completely turned my life around...",9.0,"November 27, 2016",37


In [482]:
df_label.head()

,Unnamed: 0,drugName,condition,review,rating,date,usefulCount
0,163740,Mirtazapine,Depression,"""I&#039;ve tried a few antidepressants over th...",10.0,"February 28, 2012",22
1,206473,Mesalamine,"Crohn's Disease, Maintenance","""My son has Crohn&#039;s disease and has done ...",8.0,"May 17, 2009",17
2,159672,Bactrim,Urinary Tract Infection,"""Quick reduction of symptoms""",9.0,"September 29, 2017",3
3,39293,Contrave,Weight Loss,"""Contrave combines drugs that were used for al...",9.0,"March 5, 2017",35
4,97768,Cyclafem 1 / 35,Birth Control,"""I have been on this birth control for one cyc...",9.0,"October 22, 2015",4


In [10]:
df.shape

(161297, 7)

# Data Cleaning

In [11]:
df.describe()

,Unnamed: 0,rating,usefulCount
count,161297.000000,161297.000000,161297.000000
mean,115923.585305,6.994377,28.004755
std,67004.445170,3.272329,36.403742
min,2.000000,1.000000,0.000000
25%,58063.000000,5.000000,6.000000
50%,115744.000000,8.000000,16.000000
75%,173776.000000,10.000000,36.000000
max,232291.000000,10.000000,1291.000000


In [12]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 161297 entries, 0 to 161296
Data columns (total 7 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   Unnamed: 0   161297 non-null  int64  
 1   drugName     161297 non-null  object 
 2   condition    160398 non-null  object 
 3   review       161297 non-null  object 
 4   rating       161297 non-null  float64
 5   date         161297 non-null  object 
 6   usefulCount  161297 non-null  int64  
dtypes: float64(1), int64(2), object(4)
memory usage: 8.6+ MB


In [13]:
df.drugName.value_counts()

Levonorgestrel                       3657
Etonogestrel                         3336
Ethinyl estradiol / norethindrone    2850
Nexplanon                            2156
Ethinyl estradiol / norgestimate     2117
                                     ... 
Phentride                               1
NovoLog FlexPen                         1
Collagenase                             1
Opsumit                                 1
Atrovent HFA                            1
Name: drugName, Length: 3436, dtype: int64

In [28]:
df[df['condition'].isnull()]

,Unnamed: 0,drugName,condition,review,rating,date,usefulCount
30,51452,Azithromycin,NaN,"""Very good response. It is so useful for me. """,10.0,"August 18, 2010",1
148,61073,Urea,NaN,"""Accurate information.""",10.0,"July 2, 2011",13
488,132651,Doxepin,NaN,"""So far so good. Good for me and I can take it...",10.0,"October 20, 2010",25
733,44297,Ethinyl estradiol / norgestimate,NaN,"""I haven&#039;t been on it for a long time and...",8.0,"January 24, 2011",1
851,68697,Medroxyprogesterone,NaN,"""I started the shot in July 2015 and ended in ...",6.0,"March 23, 2017",1
...,...,...,...,...,...,...,...
160468,117817,Multivitamin with minerals,NaN,"""Severe hives itching after taking for 6 months""",5.0,"November 15, 2015",0
160500,657,Medroxyprogesterone,NaN,"""I am 18 and I have been using the shot for 8 ...",6.0,"November 20, 2011",2
160689,104844,Ethinyl estradiol / levonorgestrel,NaN,"""I&#039;ve been on Loseasonique for about 2 we...",10.0,"April 13, 2010",3
160752,41252,Acetaminophen / oxycodone,NaN,"""This is my third day using this pain medicine...",10.0,"December 13, 2010",4


In [35]:
df.groupby(['drugName','condition']).first()

,,Unnamed: 0,review,rating,date,usefulCount
drugName,condition,,,,,
A + D Cracked Skin Relief,Bacterial Skin Infection,163935,"""I have severe cracked skin on my hands. I&#0...",10.0,"December 7, 2009",6
A / B Otic,Otitis Media,131173,"""It numbs the pain. It makes my ear feel heavi...",10.0,"September 23, 2009",20
Abacavir / dolutegravir / lamivudine,HIV Infection,70738,"""I have only been on it for 3 days now after b...",5.0,"January 11, 2016",8
Abacavir / lamivudine / zidovudine,HIV Infection,117239,"""I was diagnosed back in 2008 and have been on...",9.0,"September 6, 2014",1
Abatacept,Juvenile Idiopathic Arthritis,76217,"""My daughter started monthly Orencia infusions...",7.0,"May 23, 2014",16
...,...,...,...,...,...,...
depo-subQ provera 104,Birth Control,112446,"""Not only did I gain ten pounds in just a few ...",1.0,"December 19, 2016",0
ella,Emergency Contraception,163096,"""Hey,\r\r\nSo I took the ellaOne about 7h afte...",9.0,"September 13, 2016",10
femhrt,0</span> users found this comment helpful.,53297,"""I was on femHRT for four months and had BV (b...",1.0,"November 29, 2010",0


In [47]:
df.groupby(['drugName'])['condition'].count().sort_values(ascending=False).head(10)

drugName
Levonorgestrel                        3631
Etonogestrel                          3321
Ethinyl estradiol / norethindrone     2750
Nexplanon                             2156
Ethinyl estradiol / norgestimate      2033
Ethinyl estradiol / levonorgestrel    1809
Phentermine                           1538
Sertraline                            1353
Escitalopram                          1287
Mirena                                1242
Name: condition, dtype: int64

In [49]:
df.groupby(['drugName'])['Unnamed: 0'].count().sort_values(ascending=False).head(10)

drugName
Levonorgestrel                        3657
Etonogestrel                          3336
Ethinyl estradiol / norethindrone     2850
Nexplanon                             2156
Ethinyl estradiol / norgestimate      2117
Ethinyl estradiol / levonorgestrel    1888
Phentermine                           1543
Sertraline                            1360
Escitalopram                          1292
Mirena                                1242
Name: Unnamed: 0, dtype: int64

In [52]:
df.groupby(['drugName', 'condition'])['condition'].count().sort_values(ascending=False).head(20)

drugName                                                condition              
Etonogestrel                                            Birth Control              3314
Ethinyl estradiol / norethindrone                       Birth Control              2337
Nexplanon                                               Birth Control              2149
Levonorgestrel                                          Birth Control              2129
Ethinyl estradiol / levonorgestrel                      Birth Control              1600
Ethinyl estradiol / norgestimate                        Birth Control              1580
Levonorgestrel                                          Emergency Contraception    1221
Phentermine                                             Weight Loss                1211
Implanon                                                Birth Control              1095
Miconazole                                              Vaginal Yeast Infection     997
Mirena                                  

In [86]:
df_grouped_drug_cond = df.groupby(['drugName', 'condition'])['condition'].count()

In [87]:
df_grouped_drug_cond = pd.DataFrame(df_grouped_drug_cond)

In [103]:
df_grouped_drug_cond['condition'].sort_values(ascending = False)

drugName                            condition                
Etonogestrel                        Birth Control                3314
Ethinyl estradiol / norethindrone   Birth Control                2337
Nexplanon                           Birth Control                2149
Levonorgestrel                      Birth Control                2129
Ethinyl estradiol / levonorgestrel  Birth Control                1600
                                                                 ... 
Keytruda                            Melanoma                        1
                                    Non-Small Cell Lung Cance       1
Kineret                             Schnitzler Syndrome             1
Kionex                              Hyperkalemia                    1
femhrt                              Postmenopausal Symptoms         1
Name: condition, Length: 8490, dtype: int64

In [108]:
# publishing the list for manual assessment

df_grouped_drug_cond.to_csv(r'C:\Users\nishantjairath\Downloads\check.csv' )

In [131]:
# assigning freq for each condition of drugs

df_grouped_drug_cond_unique = df.groupby(['drugName'])['condition'].apply(lambda x: x.mode()).reset_index(name='val')

In [137]:
# keeping only the highest freq value (mode)

df_grouped_drug_cond_unique = df_grouped_drug_cond_unique[df_grouped_drug_cond_unique['level_1'] == 0]

In [152]:
#creating a dictionary containing drug name and mode of respective conditions

dict_drug = {}

for i in df_grouped_drug_cond_unique.iterrows():
    dict_drug[i[1][0]] = i[1][2]
    #dict_drug.value = i[1][2]

In [375]:
dict_drug

{'A + D Cracked Skin Relief': 'Bacterial Skin Infection',
 'A / B Otic': 'Otitis Media',
 'Abacavir / dolutegravir / lamivudine': 'HIV Infection',
 'Abacavir / lamivudine / zidovudine': 'HIV Infection',
 'Abatacept': 'Rheumatoid Arthritis',
 'Abilify': 'Bipolar Disorde',
 'Abilify Discmelt': 'Bipolar Disorde',
 'Abilify Maintena': 'Schizophrenia',
 'Abiraterone': 'Prostate Cance',
 'AbobotulinumtoxinA': 'Cervical Dystonia',
 'Abraxane': 'Breast Cance',
 'Abreva': 'Herpes Simplex',
 'Absorbine Jr.': 'Rheumatoid Arthritis',
 'Absorica': 'Acne',
 'Acamprosate': 'Alcohol Dependence',
 'Acanya': 'Acne',
 'Acarbose': 'Diabetes, Type 2',
 'Accolate': 'Asthma, Maintenance',
 'Accupril': 'Heart Failure',
 'Accutane': 'Acne',
 'Acebutolol': 'High Blood Pressure',
 'Acetaminophen': 'Pain',
 'Acetaminophen / aspirin': 'Back Pain',
 'Acetaminophen / aspirin / caffeine': 'Migraine',
 'Acetaminophen / aspirin / caffeine / salicylamide': 'Migraine',
 'Acetaminophen / butalbital': 'Headache',
 'Acetami

In [376]:
# replacing nan and irrelevant condition values with dictionary values

df2 = df.copy()

In [377]:
df2['condition'].isnull().sum()

899

In [378]:
df2['condition'] = df2['condition'].fillna(df2['drugName'].map(dict_drug))

In [379]:
df2[df2['condition'].isnull()]

,Unnamed: 0,drugName,condition,review,rating,date,usefulCount
65388,176065,Acetaminophen / dextromethorphan / doxylamine,NaN,"""Worked great to relieve symptoms of a cold en...",10.0,"January 23, 2016",0
122586,119730,Dalteparin,NaN,"""Information was clear to understand.""",10.0,"May 14, 2008",4
124616,58996,Methenamine / sodium acid phosphate,NaN,"""It has been the only thing that has worked fo...",10.0,"July 12, 2012",6
128655,11984,Magnesium sulfate,NaN,"""Its an awesome medicine. Gives pure relief in...",10.0,"May 22, 2016",6
146058,173788,Fluvastatin,NaN,"""Lescol lowered my LDL cholesterol from 270 to...",9.0,"August 31, 2008",25


In [380]:
# dropping the above 5 rows 

df2 = df2.dropna()

In [381]:
dict_updated = {'Amikacin':'Urinary Tract Infection', 'Atropine': 'Diarrhea, Acute', 'Dimethyl sulfoxide': 'Multiple Sclerosis'}  

In [382]:
# Updating dictionary 

dict_drug.update(dict_updated)

In [383]:

# Replace 'Not Listed / Othe' with dictionary values

for i in df2.index:
    if (df2['condition'][i] == 'Not Listed / Othe'):
        df2['condition'][i] = dict_drug[df2['drugName'][i]]
#        df2['condition'][i].replace(df2['drugName'][i]) = dict_drug.keys().values()


In [384]:
df2['condition'].value_counts()

Birth Control                           29201
Depression                               9139
Pain                                     6222
Anxiety                                  5976
Acne                                     5602
                                        ...  
Primary Hyperaldosteronism Diagnosis        1
Hyperbilirubinemia                          1
Coccidioidomycosis                          1
Dissociative Identity Disorde               1
Keratitis                                   1
Name: condition, Length: 884, dtype: int64

In [385]:
# remove special characters in condition column

for ind in df2.index:
    df2['condition'][ind] = re.sub('[^a-zA-Z0-9 \n\.]', '', df2['condition'][ind])

In [386]:
df2.head()

,Unnamed: 0,drugName,condition,review,rating,date,usefulCount
0,206461,Valsartan,Left Ventricular Dysfunction,"""It has no side effect, I take it in combinati...",9.0,"May 20, 2012",27
1,95260,Guanfacine,ADHD,"""My son is halfway through his fourth week of ...",8.0,"April 27, 2010",192
2,92703,Lybrel,Birth Control,"""I used to take another oral contraceptive, wh...",5.0,"December 14, 2009",17
3,138000,Ortho Evra,Birth Control,"""This is my first time using any form of birth...",8.0,"November 3, 2015",10
4,35696,Buprenorphine / naloxone,Opiate Dependence,"""Suboxone has completely turned my life around...",9.0,"November 27, 2016",37


In [387]:
# replacing 'users found this comment helpful' with mode of drugName level condition

for ind in df2.index:
    if ('users found this comment helpful' in df2['condition'][ind]):
                df2['condition'][ind] = dict_drug[df2['drugName'][ind]]

# EDA

In [388]:
df2['rating'] = df2['rating'].astype(int) 

In [389]:
df2['date'] = pd.to_datetime(df2['date'])

In [390]:
# Calculating recency

df2['recency'] = 1/(pd.Timestamp.now().normalize() - df2['date']).dt.days

In [391]:
df2.head()

,Unnamed: 0,drugName,condition,review,rating,date,usefulCount,recency
0,206461,Valsartan,Left Ventricular Dysfunction,"""It has no side effect, I take it in combinati...",9,2012-05-20,27,0.000263
1,95260,Guanfacine,ADHD,"""My son is halfway through his fourth week of ...",8,2010-04-27,192,0.000219
2,92703,Lybrel,Birth Control,"""I used to take another oral contraceptive, wh...",5,2009-12-14,17,0.000213
3,138000,Ortho Evra,Birth Control,"""This is my first time using any form of birth...",8,2015-11-03,10,0.000394
4,35696,Buprenorphine / naloxone,Opiate Dependence,"""Suboxone has completely turned my life around...",9,2016-11-27,37,0.000465


# Feature Engineering

In [392]:
df2.head()

,Unnamed: 0,drugName,condition,review,rating,date,usefulCount,recency
0,206461,Valsartan,Left Ventricular Dysfunction,"""It has no side effect, I take it in combinati...",9,2012-05-20,27,0.000263
1,95260,Guanfacine,ADHD,"""My son is halfway through his fourth week of ...",8,2010-04-27,192,0.000219
2,92703,Lybrel,Birth Control,"""I used to take another oral contraceptive, wh...",5,2009-12-14,17,0.000213
3,138000,Ortho Evra,Birth Control,"""This is my first time using any form of birth...",8,2015-11-03,10,0.000394
4,35696,Buprenorphine / naloxone,Opiate Dependence,"""Suboxone has completely turned my life around...",9,2016-11-27,37,0.000465


In [449]:
#rsc = RobustScaler()

#df2['rating_scaled'] = pd.DataFrame(rsc.fit_transform(df2['rating'].to_numpy().reshape(-1 , 1)))
df2['rating_scaled']  = df2['rating']/10 

In [433]:
df2['rating_scaled'].describe()

count    161287.000000
mean         -0.201133
std           0.654464
min          -1.400000
25%          -0.600000
50%           0.000000
75%           0.400000
max           0.400000
Name: rating_scaled, dtype: float64

In [450]:
#df2['usefulCount_scaled'] = pd.DataFrame(rsc.fit_transform(df2['usefulCount'].to_numpy().reshape(-1 , 1)))
df2['usefulCount_scaled'] = df2['usefulCount']/df2['usefulCount'].max()

In [451]:
#df2['recency_scaled'] = pd.DataFrame(rsc.fit_transform(df2['recency'].to_numpy().reshape(-1 , 1)))
df2['recency_scaled'] = df2['recency']/df2['recency'].max()

In [452]:
df2.describe().T

,count,mean,std,min,25%,50%,75%,max
Unnamed: 0,161292.0,115923.827443,67004.511052,2.000000,58062.750000,115743.500000,173774.500000,232291.000000
rating,161292.0,6.994290,3.272342,1.000000,5.000000,8.000000,10.000000,10.000000
usefulCount,161292.0,28.005369,36.404108,0.000000,6.000000,16.000000,36.000000,1291.000000
recency,161292.0,0.000361,0.000105,0.000187,0.000260,0.000372,0.000444,0.000565
rating_scaled,161292.0,0.699429,0.327234,0.100000,0.500000,0.800000,1.000000,1.000000
usefulCount_scaled,161292.0,0.021693,0.028198,0.000000,0.004648,0.012393,0.027885,1.000000
recency_scaled,161292.0,0.638456,0.185855,0.330903,0.460938,0.658727,0.786667,1.000000
score,161287.0,0.136789,1.423366,-2.939765,-0.816065,-0.015087,0.784134,41.924489


In [453]:
df2['score'] = df2['rating_scaled']+df2['usefulCount_scaled']+df2['recency_scaled']

In [446]:
#df2['final_score'] = pd.DataFrame(mmsc.fit_transform(df2['score'].to_numpy().reshape(-1 , 1)))
#df2.drop('final_score', axis = 1, inplace = True)

In [454]:
df2.head(20)

,Unnamed: 0,drugName,condition,review,rating,date,usefulCount,recency,rating_scaled,usefulCount_scaled,recency_scaled,score
0,206461,Valsartan,Left Ventricular Dysfunction,"""It has no side effect, I take it in combinati...",9,2012-05-20,27,0.000263,0.9,0.020914,0.465544,1.386458
1,95260,Guanfacine,ADHD,"""My son is halfway through his fourth week of ...",8,2010-04-27,192,0.000219,0.8,0.148722,0.388499,1.337221
2,92703,Lybrel,Birth Control,"""I used to take another oral contraceptive, wh...",5,2009-12-14,17,0.000213,0.5,0.013168,0.377399,0.890567
3,138000,Ortho Evra,Birth Control,"""This is my first time using any form of birth...",8,2015-11-03,10,0.000394,0.8,0.007746,0.696850,1.504596
4,35696,Buprenorphine / naloxone,Opiate Dependence,"""Suboxone has completely turned my life around...",9,2016-11-27,37,0.000465,0.9,0.028660,0.823256,1.751916
5,155963,Cialis,Benign Prostatic Hyperplasia,"""2nd day on 5mg started to work with rock hard...",2,2015-11-28,43,0.000398,0.2,0.033308,0.703777,0.937085
6,165907,Levonorgestrel,Emergency Contraception,"""He pulled out, but he cummed a bit in me. I t...",1,2017-03-07,5,0.000488,0.1,0.003873,0.863415,0.967288
7,102654,Aripiprazole,Bipolar Disorde,"""Abilify changed my life. There is hope. I was...",10,2015-03-14,32,0.000360,1.0,0.024787,0.638068,1.662855
8,74811,Keppra,Epilepsy,""" I Ve had nothing but problems with the Kepp...",1,2016-08-09,11,0.000442,0.1,0.008521,0.783186,0.891706
9,48928,Ethinyl estradiol / levonorgestrel,Birth Control,"""I had been on the pill for many years. When m...",8,2016-12-08,1,0.000468,0.8,0.000775,0.827489,1.628264


# Feature Selection

In [455]:
df2.columns

Index(['Unnamed: 0', 'drugName', 'condition', 'review', 'rating', 'date',
       'usefulCount', 'recency', 'rating_scaled', 'usefulCount_scaled',
       'recency_scaled', 'score'],
      dtype='object')

In [456]:
columns = [ 'drugName', 'condition','review','score']
df_train = df2[columns].copy(deep = True)

In [457]:
df_train.head()

,drugName,condition,review,score
0,Valsartan,Left Ventricular Dysfunction,"""It has no side effect, I take it in combinati...",1.386458
1,Guanfacine,ADHD,"""My son is halfway through his fourth week of ...",1.337221
2,Lybrel,Birth Control,"""I used to take another oral contraceptive, wh...",0.890567
3,Ortho Evra,Birth Control,"""This is my first time using any form of birth...",1.504596
4,Buprenorphine / naloxone,Opiate Dependence,"""Suboxone has completely turned my life around...",1.751916


In [461]:
df_train.nlargest(2, ['score'])

,drugName,condition,review,score
6716,Sertraline,Depression,"""I remember reading people&#039;s opinions, on...",2.340975
33552,Zoloft,Depression,"""I remember reading people&#039;s opinions, on...",2.340975


In [471]:
df_train['score'].quantile(0.50)

1.404782438951454

In [479]:
#len(df_train[df_train['score'] >= 1.67])
df_train['sentiment'] = '0'

for ind in df_train.index:
    
    if (df_train['score'][ind] >= 1.67):
        df_train['sentiment'][ind] = 'positive'
    elif (df_train['score'][ind] < 1.67 and df_train['score'][ind] >= 1.4):
        df_train['sentiment'][ind] = 'neutral'
    else:
        df_train['sentiment'][ind] = 'negative'

In [480]:
df_train.head(20)

,drugName,condition,review,score,sentiment
0,Valsartan,Left Ventricular Dysfunction,"""It has no side effect, I take it in combinati...",1.386458,negative
1,Guanfacine,ADHD,"""My son is halfway through his fourth week of ...",1.337221,negative
2,Lybrel,Birth Control,"""I used to take another oral contraceptive, wh...",0.890567,negative
3,Ortho Evra,Birth Control,"""This is my first time using any form of birth...",1.504596,neutral
4,Buprenorphine / naloxone,Opiate Dependence,"""Suboxone has completely turned my life around...",1.751916,positive
5,Cialis,Benign Prostatic Hyperplasia,"""2nd day on 5mg started to work with rock hard...",0.937085,negative
6,Levonorgestrel,Emergency Contraception,"""He pulled out, but he cummed a bit in me. I t...",0.967288,negative
7,Aripiprazole,Bipolar Disorde,"""Abilify changed my life. There is hope. I was...",1.662855,neutral
8,Keppra,Epilepsy,""" I Ve had nothing but problems with the Kepp...",0.891706,negative
9,Ethinyl estradiol / levonorgestrel,Birth Control,"""I had been on the pill for many years. When m...",1.628264,neutral


In [483]:
for ind in df_train.index:
    df_train['review'][ind] = re.sub('[^a-zA-Z0-9 \n\.]', '', df_train['review'][ind])

In [490]:
df_train.head()

,drugName,condition,review,score,sentiment
0,Valsartan,Left Ventricular Dysfunction,It has no side effect I take it in combination...,1.386458,negative
1,Guanfacine,ADHD,My son is halfway through his fourth week of I...,1.337221,negative
2,Lybrel,Birth Control,I used to take another oral contraceptive whic...,0.890567,negative
3,Ortho Evra,Birth Control,This is my first time using any form of birth ...,1.504596,neutral
4,Buprenorphine / naloxone,Opiate Dependence,Suboxone has completely turned my life around....,1.751916,positive


# Modelling

In [484]:
# 1. Transformer-Based Models from huggingface

!pip install transformers

In [486]:
import transformers
from transformers import pipeline

In [522]:
sentiment_pipeline = pipeline("sentiment-analysis", truncation = True)

No model was supplied, defaulted to distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.
Some layers from the model checkpoint at distilbert-base-uncased-finetuned-sst-2-english were not used when initializing TFDistilBertForSequenceClassification: ['dropout_19']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFDistilBertForSequenceCla

In [503]:
from tqdm import tqdm

In [524]:
df_train['output'] = '0'

for ind in tqdm(df_train.index):
    df_train['output'][ind] = sentiment_pipeline(df_train['review'][ind], truncation=True)

  3%|▎         | 5526/161292 [35:12<16:32:37,  2.62it/s]  


KeyboardInterrupt: 

In [523]:
df_train['output'] = df_train['review'].apply(sentiment_pipeline)

KeyboardInterrupt: 

# 2. LSTM (Long Short Term Memory) model

In [ ]:
import nltk
from textblob import Word
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from keras.model import Sequential
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer

In [ ]:
token = RegexpTokenizer(r'[a-zA-Z0-9]+')
cv = CountVectorizer(stop_words = 'english', ngram_range = (1,1), tokenizer = token.tokenize)
text_counts = cv.fit_transform(df_train['reviews'])

X_train, X_test, Y_train, Y_test = train_test_split(text_counts, df_train['sentiment'], test_size = 0.25, random_state = 5)


In [ ]:
# Pre-processing the text

def cleaning(df, stop_words):
    df['review'] = df['review'].apply(lambda x: ''.join(x.lower() for x in x.split()))
    df['review'] = df['review'].str.replace('d', '')
    df['review'] = df['review'].apply(lambda x: ''.join(x for x in x.split() if x not in stop_words))
    df['review'] = df['review'].apply(lambda x: ''.join([Word(x).lemmatize() for x in x.split()]))
    return df

In [ ]:
stop_words = stopwords.words('english')
data_cleaned = cleaning(df_train, stop_words)

In [ ]:
# Generating embeddings using tokenizer

tokenizer = Tokenizer(num_words = 500, split = '')
tokenizer.fit_on_texts(df_train['review'].values)
X = tokenizer.texts_to_sequences(df_train['review'].values)
X = pad_sequences(X)

In [ ]:
# Model Building

model = Sequential()
model.add(Embedding(500, 120, input_length = X.shape[1]))
model.add(SpatialDropout1D(0.4))
model.add(LSTM(704, dropout = 0.2, recurrent_dropout = 0.2))
model.add(Dense(352, activation = 'LeakyReLU'))
model.add(Dense(3, activation = 'softmax'))

In [ ]:
model.compile(loss = 'categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
print(model.summary())

In [ ]:
model.fit(X_train, Y_train, epochs = 20, batch_sizes = 32, verbose = 1)
model.evaluate(X_test, Y_test)

# Prediction

In [408]:
# Data reference

'Felix Gräßer, Surya Kallumadi, Hagen Malberg, and Sebastian Zaunseder. 2018. Aspect-Based Sentiment Analysis of Drug Reviews Applying Cross-Domain and Cross-Data Learning. In Proceedings of the 2018 International Conference on Digital Health (DH '18). ACM, New York, NY, USA, 121-125. DOI'

SyntaxError: invalid syntax (<ipython-input-408-2e93f4244806>, line 3)